# Project Title
### Data Engineering Capstone Project


#### Project Summary
Creating a data lake for immigration activity data in order to be able to make analytical queries on the star schema to be created. 
Also we will be able to make ad hoc queries in the raw data (schema-on read) since raw data reside on the same space. 

**! ! ! The assessor can run this whole project because the data lake is created locally in parquet files ! ! !**

**All the assessor needs to do is to go through this Jupiter notebook.**

The project follows the follow steps:
* Step 1: Scope the Project and Gather Data
* Step 2: Explore and Assess the Data
* Step 3: Define the Data Model
* Step 4: Run ETL to Model the Data
* Step 5: Complete Project Write Up

In [1]:
# Do all imports and installs here
import os
import pandas as pd
from datetime import datetime, timedelta
from pyspark.sql import SparkSession
from pyspark.sql.types import StructType, IntegerType, TimestampType, StringType, StructField
from pyspark.sql.functions import udf, col, date_add

from helpful_functions import *

In [2]:
# Instatiate Spark session
spark = create_spark_session()

In [3]:
# set pandas dataframes dispay columns to unlimited
pd.set_option('max_columns', None)

### Step 1: Scope the Project and Gather Data

#### Scope 

We are creating a data warehouse as part of a data lake where the analytics team would be able to analyze immigration movement and specifically which points of entry(e.g. which airports) are mostly involved. With that in mind the data model is designed in a way that supports flexibility on analytical queries. Queries like "Top 10 points of entry that immigrants enter a country".

We make it easy and efficient for the team to write these queries as there is at most one JOIN operation included (advantage of the denormalized form) to get a piece of information (e.g. join country code with countries dimension to find a country's name).

Our data warehouse will be written into parquet files in order to take advantage of their columnar storage for better performance on fetching column data. Also we gain storage efficiency as compressed columnar files take less space.

In addition we keep the raw data in the same space. This serves well the data scientists and machine learning engineers since they have a lot of flexibility on reading the data however they want and they can run ad-hoc queries on the raw data directly (schema-on-read). Therefore our data lake can serve Data Scientists and ML Engineers along with business analysts in comparison with a traditional data warehouse which could only serve business analysts.

#### Describe and Gather Data 
Describe the data sets you're using. Where did it come from? What type of information is included? 
##### Datasets:
**Immigration data**: Provided by Udacity. This data comes from the [US National Tourism and Trade Office](https://www.trade.gov/national-travel-and-tourism-office). They contain immigration information for individuals e.g. the mode of entry (sea, air, land).

**Airport data**: Provided by Udacity. This data contains airport codes and corresponding cities and they come from [datahub.io](https://datahub.io/core/airport-codes#data).

**I94 Labels**: Provided by Udacity. These data come from I94_SAS_Labels_Descriptions.SAS file and they contain codes and their descriptions (e.g. city codes and the actual city names). In the following block of code we parse the content of this file in order to create csv datasets, for the codes we choose in order to read them as dataframes.

*****The Immigration Data will be used to create our facts table while the other two datasets will be used for the dimensions.*****

In [4]:
# create csv files out of the codes and save them in label_data_sources folder
# I choose to create csv files for only state, country and mode of entry codes
#(labelled with 'i94addrl', 'i94cntyl' and 'i94model' respectively ).
create_data_sources_from_label_file('I94_SAS_Labels_Descriptions.SAS', 'label_data_sources', 'i94addrl', 'i94cntyl', 'i94model') 

In [5]:
# read immigration data
immigration_data = './sas_data'
immigration_df = spark.read.load(immigration_data)

immigration_df.createOrReplaceTempView("immigration_staging")
spark.sql("""
SELECT *
FROM immigration_staging
LIMIT 5
"""
).toPandas()

,cicid,i94yr,i94mon,i94cit,i94res,i94port,arrdate,i94mode,i94addr,depdate,i94bir,i94visa,count,dtadfile,visapost,occup,entdepa,entdepd,entdepu,matflag,biryear,dtaddto,gender,insnum,airline,admnum,fltno,visatype
0,5748517.0,2016.0,4.0,245.0,438.0,LOS,20574.0,1.0,CA,20582.0,40.0,1.0,1.0,20160430,SYD,None,G,O,None,M,1976.0,10292016,F,None,QF,9.495387e+10,00011,B1
1,5748518.0,2016.0,4.0,245.0,438.0,LOS,20574.0,1.0,NV,20591.0,32.0,1.0,1.0,20160430,SYD,None,G,O,None,M,1984.0,10292016,F,None,VA,9.495562e+10,00007,B1
2,5748519.0,2016.0,4.0,245.0,438.0,LOS,20574.0,1.0,WA,20582.0,29.0,1.0,1.0,20160430,SYD,None,G,O,None,M,1987.0,10292016,M,None,DL,9.495641e+10,00040,B1
3,5748520.0,2016.0,4.0,245.0,438.0,LOS,20574.0,1.0,WA,20588.0,29.0,1.0,1.0,20160430,SYD,None,G,O,None,M,1987.0,10292016,F,None,DL,9.495645e+10,00040,B1
4,5748521.0,2016.0,4.0,245.0,438.0,LOS,20574.0,1.0,WA,20588.0,28.0,1.0,1.0,20160430,SYD,None,G,O,None,M,1988.0,10292016,M,None,DL,9.495639e+10,00040,B1


In [6]:
# Read the airport data here
airport_data = 'airport-codes_csv.csv'
airport_df = spark.read.csv(airport_data, header=True)

airport_df.createOrReplaceTempView("airports_staging")
spark.sql("""
SELECT *
FROM airports_staging
LIMIT 5
"""
).toPandas()

,ident,type,name,elevation_ft,continent,iso_country,iso_region,municipality,gps_code,iata_code,local_code,coordinates
0,00A,heliport,Total Rf Heliport,11,NA,US,US-PA,Bensalem,00A,None,00A,"-74.93360137939453, 40.07080078125"
1,00AA,small_airport,Aero B Ranch Airport,3435,NA,US,US-KS,Leoti,00AA,None,00AA,"-101.473911, 38.704022"
2,00AK,small_airport,Lowell Field,450,NA,US,US-AK,Anchor Point,00AK,None,00AK,"-151.695999146, 59.94919968"
3,00AL,small_airport,Epps Airpark,820,NA,US,US-AL,Harvest,00AL,None,00AL,"-86.77030181884766, 34.86479949951172"
4,00AR,closed,Newport Hospital & Clinic Heliport,237,NA,US,US-AR,Newport,None,None,None,"-91.254898, 35.6087"


In [7]:
# Read the state data here
state_data = 'label_data_sources/addr.csv'
schema = StructType([
    StructField("state_code", StringType()),
    StructField("state", StringType())
])

state_df = spark.read.csv(state_data, header=False, schema=schema)
state_df.createOrReplaceTempView("state_codes")
spark.sql("""
SELECT *
FROM state_codes
"""
).toPandas().head(5)


,state_code,state
0,AL,ALABAMA
1,AK,ALASKA
2,AZ,ARIZONA
3,AR,ARKANSAS
4,CA,CALIFORNIA


In [8]:
# Read the entry mode data here

mode_data = 'label_data_sources/mode.csv'
schema = StructType([
    StructField("mode_code", IntegerType()),
    StructField("mode", StringType())
])

mode_df = spark.read.csv(mode_data, header=False, schema=schema)
mode_df.createOrReplaceTempView("mode_codes")
spark.sql("""
SELECT *
FROM mode_codes
"""
).toPandas()


,mode_code,mode
0,1,Air
1,2,Sea
2,3,Land
3,9,Not reported


In [9]:
# Read the country data here

country_data = 'label_data_sources/cnty.csv'
schema = StructType([
    StructField("country_code", IntegerType()),
    StructField("country", StringType())
])

addr_df = spark.read.csv(country_data, header=False, schema=schema)
addr_df.createOrReplaceTempView("country_codes")
spark.sql("""
SELECT *
FROM country_codes
LIMIT 5
"""
).toPandas()


,country_code,country
0,582,"MEXICO Air Sea, and Not Reported (I-94, no lan..."
1,236,AFGHANISTAN
2,101,ALBANIA
3,316,ALGERIA
4,102,ANDORRA


### Step 2: Explore and Assess the Data
#### Explore the Data 

1. From a quick look on the immigration_df above we can see that cicid, i94yr, i94mon, i94res, i94mode, biryear, count are float so we will need to cast them as integers (some of these will also be used to join with the dimension tables on integer values)

2. We also need to check if cicid in immigration data is unique in order to ensure that it uniquely identifies an immigration movement record.

3. Check that the ***iata_code*** field in the airport codes dataset actually matches with the ***i94port*** field of the immiggration data so that we know if airport_codes.csv serves our use case. I have actually performed this check before I chose my datasets in order to ensure that this dataset can be joined with the immigration dataset so that it suits my purpose.

4. Check for null values on the fields i94res, i94mode , i94port, i94addr which will be the dimension fields in our facts table. 

5. Also check for null values and duplicated values in iata_code field of airport codes dataset.

6. Check for duplicates in country_df, state_df. 

7. Check counts before and after cleaning the data.

Below we perform the above checks:


#### Check 1 
From a quick look on the immigration_df above we can see that cicid, i94yr, i94mon, i94res, i94mode, biryear, count are float so we will need to cast them as integers (some of these will also be used to join with the dimension tables on integer values).

This check has been done above by looking the at the displayed immigration data.

#### Check 2
We also need to check if cicid in immigration data is unique in order to ensure that uniquely identifies an immigration activity record.

In [10]:
# count all the records
spark.sql("""
SELECT COUNT(*)
FROM immigration_staging
"""
).show()
# Returns 3096313

# count unique cicid values
spark.sql("""
SELECT COUNT(DISTINCT cicid)
FROM immigration_staging
"""
).show()

# The counts are the same (3096313) therefore cicid uniquely identifies an immigration record

+--------+
|count(1)|
+--------+
| 3096313|
+--------+

+---------------------+
|count(DISTINCT cicid)|
+---------------------+
|              3096313|
+---------------------+



#### Check 3
Check that the ***iata_code*** field in the airport codes dataset actually matches with the ***i94port*** field of the immigration data so that we know if airport_codes.csv serves our use case. I have actually performed this check before I choose my datasets in order to ensure that this dataset can be joined with the immigration dataset so that it suits my purpose.

In [11]:
# Getting all the iata_code values as a set
iata_codes = spark.sql("""
SELECT DISTINCT iata_code
FROM airports_staging
"""
)
iata_codes_set = set(iata_codes.rdd.flatMap(lambda x: x).collect())

# Getting all the i94port values as a set
i94port_codes = spark.sql("""
SELECT DISTINCT i94port
FROM immigration_staging
"""
)
i94port_codes_set = set(i94port_codes.rdd.flatMap(lambda x: x).collect())

# checking if there is an intersection between the two sets
len(iata_codes_set.intersection(i94port_codes_set))
# Returns 251 therefore there is intersection thus we can join i94port with iata_codes

251

#### Check 4. 
Check for null values on the fields i94res, i94mode , i94port, i94addr which will be the dimension join fields in our facts table. 

In [12]:
from pyspark.sql.functions import isnan, when, count, col

# found this piece of code in https://stackoverflow.com/questions/44627386/how-to-find-count-of-null-and-nan-values-for-each-column-in-a-pyspark-dataframe
immigration_df.select([count(when(isnan(c) | col(c).isNull(), c)).alias(c) for c in ['i94res', 'i94mode' , 'i94port', 'i94addr']]).show()

spark.sql("""
SELECT i94res, COUNT(*) AS count
FROM immigration_staging
GROUP BY i94res
ORDER BY i94res
"""
).show()

spark.sql("""
SELECT i94mode, COUNT(*) AS count
FROM immigration_staging
GROUP BY i94mode
ORDER BY i94mode
"""
).show()

spark.sql("""
SELECT i94port, COUNT(*) AS count
FROM immigration_staging
GROUP BY i94port
ORDER BY i94port
"""
).show()

spark.sql("""
SELECT i94addr, COUNT(*) AS count
FROM immigration_staging
GROUP BY i94addr
ORDER BY i94addr
"""
).show()

+------+-------+-------+-------+
|i94res|i94mode|i94port|i94addr|
+------+-------+-------+-------+
|     0|    239|      0| 152592|
+------+-------+-------+-------+

+------+------+
|i94res| count|
+------+------+
| 101.0|   929|
| 102.0|   117|
| 103.0| 15465|
| 104.0| 20796|
| 105.0|  2343|
| 107.0| 16153|
| 108.0| 24600|
| 109.0|  1983|
| 110.0| 11545|
| 111.0|185339|
| 112.0|156613|
| 113.0|  7251|
| 114.0|  5835|
| 115.0|  3922|
| 116.0| 29894|
| 117.0| 65782|
| 118.0|  1475|
| 119.0|   168|
| 120.0|  2316|
| 121.0|  1417|
+------+------+
only showing top 20 rows

+-------+-------+
|i94mode|  count|
+-------+-------+
|   null|    239|
|    1.0|2994505|
|    2.0|  26349|
|    3.0|  66660|
|    9.0|   8560|
+-------+-------+

+-------+-----+
|i94port|count|
+-------+-----+
|    5KE|    3|
|    5T6|    4|
|    ABG|   11|
|    ABQ|    3|
|    ABS|    4|
|    ADS|   15|
|    ADT|   10|
|    ADW|    4|
|    AGA|80919|
|    AGN|   11|
|    ALC|   39|
|    ANA|    1|
|    ANC|   91|
|    

#### Conclusions on check 4 above: 

We can see that some of our chosen dimensions contain null values. Also they contain weird formatted codes that definitely do not exist in the codes datasets.
Therefore we will use left joins with the dimension data when we create our facts table.
Since I want to keep all the information on the immigration data I will not remove any of the records containing null or weird codes in the above fields.
But if I wanted to "clean" these data I would use joins of the immigration data with all the dimensions, for the creation of my fact table only the values that are matched are kept in the fact table.

#### Check 5
Also check for null values and duplicated values in iata_code field of airport codes dataset.

In [13]:
spark.sql("""
SELECT iata_code, COUNT(*) AS count
FROM airports_staging
GROUP BY iata_code
having count > 1
ORDER BY iata_code
"""
).show()

+---------+-----+
|iata_code|count|
+---------+-----+
|     null|45886|
|        0|   80|
|      AHT|    2|
|      AUS|    2|
|      BCK|    2|
|      BFC|    2|
|      BVW|    2|
|      CLG|    2|
|      CMN|    2|
|      CQP|    2|
|      CSZ|    2|
|      CTR|    2|
|      DDU|    2|
|      DLR|    2|
|      DZI|    2|
|      ESP|    2|
|      GGC|    2|
|      GVA|    2|
|      HAT|    2|
|      HDB|    2|
+---------+-----+
only showing top 20 rows



#### Remarks on check 5 above: 

We can see that iata_code has null values on the iata code field.
Also there are codes that appear more than once. 
We will remove the records with null iata_codes and deduplicate the ones that appear more than once in the cleaning steps.
We do that because we want iata_code to be unique (dimension primary key).

The below steps for check 5 will be used in the cleaning process of airport_codes.

In [14]:
# We will take only the codes that appear once in the dataset and check if still the dataset is sufficient enough (i.e contains enough records to match immigration data)

# getting all the iata_codes that appear only once
non_duplicate_iata_codes = spark.sql("""
select iata_code
from
(SELECT iata_code, COUNT(*) AS count
FROM airports_staging
GROUP BY iata_code
having count = 1
)
"""
)
non_duplicate_iata_codes.createOrReplaceTempView("non_duplicate_iata_codes")

# selecting the records that their iata_codes appear only once
dedup_airports = spark.sql("""
select * 
from airports_staging
where iata_code in (select * from non_duplicate_iata_codes)
"""
)

dedup_airports.createOrReplaceTempView("dedup_airports")

In [15]:
# Checking that the iata_code field in the deduplicated airport codes dataset (dedup_airports) actually still matches with  enough records from the immigration data

iata_codes = spark.sql("""
SELECT DISTINCT iata_code
FROM dedup_airports
"""
)
iata_codes_set = set(iata_codes.rdd.flatMap(lambda x: x).collect())

# Getting all the i94port values as a set
i94port_codes = spark.sql("""
SELECT DISTINCT i94port
FROM immigration_staging
"""
)
i94port_codes_set = set(i94port_codes.rdd.flatMap(lambda x: x).collect())

# checking if there is an intersection between the two sets
len(iata_codes_set.intersection(i94port_codes_set))
# Returns 247 while the initial duplicated set returned 251, therefore there is a sufficient intersection thus we can join i94port with iata_codes after deduplication.

247

The above steps of check 5 will be used in the cleaning process of airport_codes.

#### Check 6
Check for duplicates in country_df, state_df. 


In [16]:
spark.sql("""
SELECT country_code, COUNT(*) AS count
FROM country_codes
GROUP BY country_code
HAVING count > 1
ORDER BY country_code
"""
).show()
# returns 0 records thus no duplicates

spark.sql("""
SELECT state_code, COUNT(*) AS count
FROM state_codes
GROUP BY state_code
HAVING count > 1
ORDER BY state_code
"""
).show()
# returns 0 records thus no duplicates

+------------+-----+
|country_code|count|
+------------+-----+
+------------+-----+

+----------+-----+
|state_code|count|
+----------+-----+
+----------+-----+



#### Check 7
Check counts.


In [17]:
# immigration data
spark.sql("""
SELECT COUNT(*) AS immigration_count
FROM immigration_staging
"""
).show()
# Returns 3096313

# airport data
# This is the count before cleaning/deduplicating iata_code. 
# We have to check the count after cleaning data as well to see if there are still enough records after deduplication. 
spark.sql("""
SELECT COUNT(*) AS airport_count
FROM airports_staging
"""
).show()
# Returns 55075


# immigration data
spark.sql("""
SELECT COUNT(*) AS countries_count
FROM country_codes
"""
).show()
# Returns 289

# airport data
spark.sql("""
SELECT COUNT(*) AS states_count
FROM state_codes
"""
).show()
# Returns 55


+-----------------+
|immigration_count|
+-----------------+
|          3096313|
+-----------------+

+-------------+
|airport_count|
+-------------+
|        55075|
+-------------+

+---------------+
|countries_count|
+---------------+
|            289|
+---------------+

+------------+
|states_count|
+------------+
|          55|
+------------+



#### Cleaning Steps
Document steps necessary to clean the data.

1. Cleaning Immigration data: Cast cicid, i94yr, i94mon, i94res, i94mode, biryear, count as integers. Casting arrdate into date format.

2. Cleaning airport data: Deduplicate iata_code field. Check count after deduplication.



***Cleaning Immigration data***:

In [18]:
# casting float fields to integers
clean_immigration_df = immigration_df.withColumn("cicid", col("cicid").cast(IntegerType())) \
                                    .withColumn("i94yr", col("i94yr").cast(IntegerType())) \
                                    .withColumn("i94mon", col("i94mon").cast(IntegerType())) \
                                    .withColumn("i94res", col("i94res").cast(IntegerType())) \
                                    .withColumn("i94mode", col("i94mode").cast(IntegerType())) \
                                    .withColumn("biryear", col("biryear").cast(IntegerType())) \
                                    .withColumn("count", col("count").cast(IntegerType())) 


# coverting days to date format for arrival date
get_date = udf(lambda days : datetime(1960, 1, 1) + timedelta(days=int(days)), TimestampType())
clean_immigration_df = clean_immigration_df.withColumn("arr_date", get_date("arrdate")) 

clean_immigration_df.createOrReplaceTempView("clean_immigration")
spark.sql("""
SELECT *
FROM clean_immigration
LIMIT 5
"""
).toPandas()
# Cleaning successfull


,cicid,i94yr,i94mon,i94cit,i94res,i94port,arrdate,i94mode,i94addr,depdate,i94bir,i94visa,count,dtadfile,visapost,occup,entdepa,entdepd,entdepu,matflag,biryear,dtaddto,gender,insnum,airline,admnum,fltno,visatype,arr_date
0,5748517,2016,4,245.0,438,LOS,20574.0,1,CA,20582.0,40.0,1.0,1,20160430,SYD,None,G,O,None,M,1976,10292016,F,None,QF,9.495387e+10,00011,B1,2016-04-30
1,5748518,2016,4,245.0,438,LOS,20574.0,1,NV,20591.0,32.0,1.0,1,20160430,SYD,None,G,O,None,M,1984,10292016,F,None,VA,9.495562e+10,00007,B1,2016-04-30
2,5748519,2016,4,245.0,438,LOS,20574.0,1,WA,20582.0,29.0,1.0,1,20160430,SYD,None,G,O,None,M,1987,10292016,M,None,DL,9.495641e+10,00040,B1,2016-04-30
3,5748520,2016,4,245.0,438,LOS,20574.0,1,WA,20588.0,29.0,1.0,1,20160430,SYD,None,G,O,None,M,1987,10292016,F,None,DL,9.495645e+10,00040,B1,2016-04-30
4,5748521,2016,4,245.0,438,LOS,20574.0,1,WA,20588.0,28.0,1.0,1,20160430,SYD,None,G,O,None,M,1988,10292016,M,None,DL,9.495639e+10,00040,B1,2016-04-30



***Cleaning Airport data***:

In [19]:
# getting all the iata_codes that appear only once
non_duplicate_iata_codes = spark.sql("""
select iata_code
from
(SELECT iata_code, COUNT(*) AS count
FROM airports_staging
GROUP BY iata_code
having count = 1
)
"""
)
non_duplicate_iata_codes.createOrReplaceTempView("non_duplicate_iata_codes")

# selecting the records that their iata_codes appear only once
clean_airports_df = spark.sql("""
select * 
from airports_staging
where iata_code in (select * from non_duplicate_iata_codes)
"""
)

clean_airports_df.count()
# Returns 8975. Sufficient amount of records for a dimension.

8975

### Step 3: Define the Data Model
#### 3.1 Conceptual Data Model
Data model defined in model.png

The facts table is created by the immigration data that reside inside the sas_data folder.<br>
The countries_dim, modes_dim and states_dim dimension folders are created from I94_SAS_Labels_Descriptions.SAS file after it is parsed into csv files in the label_data_sources folder.<br>
The airports_dim dimesion is created from the airport_codes.csv file.

#### 3.2 Mapping Out Data Pipelines
List the steps necessary to pipeline the data into the chosen data model

1. Create csv files for countries, states and modes out from I94_SAS_Labels_Descriptions.SAS file and save them label_data_sources folder.

2. Read the immigration data from the sas_data folder and the airport_codes.csv to spark dataframes.

3. Read the countries, modes and states data from the label_data_sources folder into spark dataframes.

4. Clean the immigration dataframe and the airport dataframe and save the cleaned results into new dataframes (i.e. clean dataframes).

5. Create the dimension tables: Write the countries, modes and states dataframes into parquet files. Write the clean airports dataframe into a parquet files. (Lets call these dataframes dimension dataframes)

6. Create the fact table: Left join the clean immigration dataframe with all the dimension dataframes and write the result in a paquet file.

### Step 4: Run Pipelines to Model the Data 
#### 4.1 Create the data model
Build the data pipelines to create the data model.
At this point we need to **RESTART THE KERNEL** of the notebook so that we can run the pipeline from the very begining.

**! ! ! RESTART THE KERNEL ! ! !**

In [1]:
# import all the function for the etl
from helpful_functions import *

In [2]:
# Instatiate Spark session
spark = create_spark_session()

In [3]:
# create csv files out of the I94_SAS_Labels_Descriptions.SAS file and save them in label_data_sources folder
create_data_sources_from_label_file('I94_SAS_Labels_Descriptions.SAS', 'label_data_sources', 'i94addrl', 'i94cntyl', 'i94model') 

In [4]:
# read immigration data to a spark dataframe
immigration_data = './sas_data'

immigration_df = spark.read.load(immigration_data)

In [5]:
# read airport data to a spark dataframe
airport_data = 'airport-codes_csv.csv'

airport_df = spark.read.csv(airport_data, header=True)

In [6]:
# Read the state data to a spark dataframe
state_data = 'label_data_sources/addr.csv'

state_df = read_state_data(spark, state_data)

In [7]:
# Read the country data to a spark dataframe
country_data = 'label_data_sources/cnty.csv'

country_df = read_country_data(spark, country_data)

In [8]:
# Read the entry mode data to a spark dataframe
mode_data = 'label_data_sources/mode.csv'

mode_df = read_mode_data(spark, mode_data)

In [9]:
# Clean immigration_df
clean_immigration_df = clean_immigration(immigration_df)

In [10]:
# Clean airport_df
clean_airports_df = clean_airports(spark, airport_df)

In [11]:
# Create dimensions

# create countries_dim
country_df.write.option("header",True).mode("overwrite").parquet("star-schema/countries_dim")

# create modes_dim
state_df.write.option("header",True).mode("overwrite").parquet("star-schema/states_dim")

# create states_dim
mode_df.write.option("header",True).mode("overwrite").parquet("star-schema/modes_dim")

# create airports_dim
final_airports_df = create_airports_dim(spark, clean_airports_df)
final_airports_df.write.option("header",True).mode("overwrite").parquet("star-schema/airports_dim")



In [12]:
# Create fact table
immigration_fact_df = create_immigration_fact(spark, clean_immigration_df)
immigration_fact_df.write.option("header",True).mode("overwrite").parquet("star-schema/immigrations_fact")


#### 4.2 Data Quality Checks
Explain the data quality checks you'll perform to ensure the pipeline ran as expected. These could include:
 
 1. Check the counts of our fact and dimension tables and compare them with the counts of the spark dataframes that were created from reading the data initially.
 2. Check for the uniqueness of our primary keys in the fact and dimensions tables.
 
Run Quality Checks

In [13]:
# read the parquet files into spark dataframes

immigration_fact = spark.read.parquet("star-schema/immigrations_fact")
immigration_fact.createOrReplaceTempView("immigration_fact")

states_dim = spark.read.parquet("star-schema/states_dim")
states_dim.createOrReplaceTempView("states_dim")

modes_dim = spark.read.parquet("star-schema/modes_dim")
modes_dim.createOrReplaceTempView("modes_dim")

countries_dim = spark.read.parquet("star-schema/countries_dim")
countries_dim.createOrReplaceTempView("countries_dim")

airports_dim = spark.read.parquet("star-schema/airports_dim")
airports_dim.createOrReplaceTempView("airports_dim")

Below we check the counts of our fact and dimension tables and compare them with the counts of the spark dataframes that were created from reading the data initially.

In [14]:
spark.sql("""
SELECT count(*)
FROM immigration_fact
"""
).show()
# Returns 3096313 which is like the initial dataset count (see check 7 in Explore and Assess the Data Section)
# Therefore our we kept all the information from the initial dataset as we wanted
# Quality check: PASS

+--------+
|count(1)|
+--------+
| 3096313|
+--------+



In [15]:
spark.sql("""
SELECT count(*)
FROM countries_dim
"""
).show()
# Returns 289 which is like the initial dataset count (see check 7 in Explore and Assess the Data Section)
# Quality check: PASS

+--------+
|count(1)|
+--------+
|     289|
+--------+



In [16]:
spark.sql("""
SELECT count(*)
FROM modes_dim
"""
).show()
# Returns 4 which is like the initial dataset count (We know that initial dataser just  had 4 records)
# Quality check: PASS

+--------+
|count(1)|
+--------+
|       4|
+--------+



In [17]:
spark.sql("""
SELECT count(*)
FROM states_dim
"""
).show()
# Returns 55 which is like the initial dataset count (see check 7 in Explore and Assess the Data Section)
# Quality check: PASS

+--------+
|count(1)|
+--------+
|      55|
+--------+



In [18]:
spark.sql("""
SELECT count(*)
FROM airports_dim
"""
).show()
# Returns 8975 which is like the cleaned airports dataframe(see Cleaning Steps Section)
# Quality check: PASS

+--------+
|count(1)|
+--------+
|    8975|
+--------+



Below we check for the uniqueness of our primary keys in the fact and dimensions tables.

In [19]:
spark.sql("""
SELECT COUNT(DISTINCT cicid)
FROM immigration_fact
"""
).show()
# Returns 3096313 which is the same as the number of records in the table therefore it is unique
# Quality check: PASS

+---------------------+
|count(DISTINCT cicid)|
+---------------------+
|              3096313|
+---------------------+



In [20]:
spark.sql("""
SELECT COUNT(DISTINCT code)
FROM countries_dim
"""
).show()
# Returns 289 which is the same as the number of records in the table therefore it is unique
# Quality check: PASS

+--------------------+
|count(DISTINCT code)|
+--------------------+
|                 289|
+--------------------+



In [21]:
spark.sql("""
SELECT COUNT(DISTINCT code)
FROM modes_dim
"""
).show()
# Returns 4 which is the same as the number of records in the table therefore it is unique
# Quality check: PASS

+--------------------+
|count(DISTINCT code)|
+--------------------+
|                   4|
+--------------------+



In [22]:
spark.sql("""
SELECT COUNT(DISTINCT code)
FROM states_dim
"""
).show()
# Returns 55 which is the same as the number of records in the table therefore it is unique
# Quality check: PASS

+--------------------+
|count(DISTINCT code)|
+--------------------+
|                  55|
+--------------------+



In [23]:
spark.sql("""
SELECT COUNT(DISTINCT iata_code)
FROM airports_dim
"""
).show()
# Returns 8975 which is the same as the number of records in the table therefore it is unique
# Quality check: PASS

+-------------------------+
|count(DISTINCT iata_code)|
+-------------------------+
|                     8975|
+-------------------------+



#### 4.3 Data dictionary 

**immigration_fact** <br>
 |-- cicid: unique id <br>
 |-- year: year <br>
 |-- month: month in number format <br>
 |-- country_code: country code<br>
 |-- airport_code:  airport code <br>
 |-- arrival_date: date of arrival in the country<br>
 |-- mode_code: code for mode of entry<br>
 |-- state: state code<br>
 |-- count: value 1 for all records (used for statistics)<br>
 |-- birth_year: year of birth <br>
 |-- gender: gender<br>
 |-- airline: code for airline<br>
 |-- visa_type: code for visa type


**states_dim**<br>
 |-- code: 194sas label state code<br>
 |-- state: state name


**modes_dim**<br>
 |-- code: 194sas label travel mode code<br>
 |-- mode: travel mode of entry

**countries_dim**<br>
 |-- code: 194sas label country code<br>
 |-- country: name of country

**airports_dim**<br>
 |-- iata_code: airport code<br>
 |-- identifier: identifier for the airport<br>
 |-- type: type of airport<br>
 |-- name: name of airport<br>
 |-- elevation_ft: elevation <br>
 |-- continent: continent code <br>
 |-- country: country code<br>
 |-- region: region code<br>
 |-- municipality: municipality<br>

#### Step 5: Complete Project Write Up
* Clearly state the rationale for the choice of tools and technologies for the project.
* Propose how often the data should be updated and why.
* Write a description of how you would approach the problem differently under the following scenarios:
 * The data was increased by 100x.
 * The data populates a dashboard that must be updated on a daily basis by 7am every day.
 * The database needed to be accessed by 100+ people.

**Clearly state the rationale for the choice of tools and technologies for the project.**

* I choose Spark in order to take advantage of it's parrallellization of tasks during transfromations.
* I chose to write my data in parquet files in order to take advantage of their columnar storage for better performance on fetching column data. Also we gain storage efficiency as compressed columnar files take less space.
* I used pandas for their eye-friendly displaying of dataframes in the Jupiter Notebooks to make the exploration of data easier.

**Propose how often the data should be updated and why.**<br>
The data should be updated every day since the analytics team may have questions such as "Which day of the last week did the most immigrants arrive?"

**If the data was incresed by 100x** then I would use a Spark cluster and add more nodes to the cluster as data increased.

**If the data populates a dashboard that must be updated on a daily basis by 7am every day** then I would use Apache airflow in order to take advantage of its scheduling feature.

**If the database needed to be accessed by 100+ people** then I would also write another pipeline step that creates and populates the data model in AWS Redshift since it can handle many connections and users as indicated in [AWS website](https://docs.aws.amazon.com/redshift/latest/mgmt/amazon-redshift-limits.html).

Finishing with the example query that is mentioned in the scope:

In [26]:
# Top 10 points of entry that immigrants enter a country

spark.sql("""
    SELECT a.name, count(*) as count
    FROM immigration_fact im
    INNER JOIN airports_dim a on a.iata_code = im.airport_code
    GROUP BY a.name
    ORDER BY count DESC
    LIMIT 10
    """
).show()

+--------------------+------+
|                name| count|
+--------------------+------+
|Miami Internation...|343941|
|Murtala Muhammed ...|310163|
|San Fernando Airport|152586|
|Orlando Executive...|149195|
|   Lakefront Airport|136122|
|William P Hobby A...|101481|
|Hartsfield Jackso...| 92579|
|  Al Massira Airport| 80919|
|   Dallas Love Field| 71809|
|General Edward La...| 57354|
+--------------------+------+

